In [1]:
import cvxpy as cp
import numpy as np

priority_ratio = 100
v = cp.Variable((1,))

# out_enable = cp.Variable(1, boolean=True)
num = 10
# cp.min(in_enable, in_mod) == 
# BIG_NUMBER * (in_enable-in_sub - (out_enable - out_sub))
# (in_enable-in_sub) >= 0
# (out_enable-out_sub) >= 0

# SWITCH    MOD     EQL
# 0         0       0
# 1         -0.5    0.5
# -1        0.5     -0.5




# 1A + 3B + 0.5C = 1 recipe completion
# coeffs = np.array([1, 3, 0.5])
# constraints = [cp.min(cp.multiply(1/coeffs, v)) >= 1]

# switch = cp.Variable(1, integer=True)
# modifier = cp.Variable(1, bounds=[-1, 1])
# switch_ranges = [cp.max(switch[i]+modifier[i], -1*(switch[i]+modifier[i]))  >= 0 for i in range(switch.shape[0])]
# diff_terms = cp.maximum(switch+modifier, -1*(switch+modifier)) 
# low_bounds = diff_terms >= 0
# high_bounds = diff_terms  <= 1

# constraints = [low_bounds, high_bounds,
#             100* (switch[0]+modifier[0]) == 0.1
#         ]

num_vars = 1
in_enable = cp.Variable(num_vars, boolean=True)
in_sub = cp.Variable(num_vars, bounds=[0, 1])
out_enable = cp.Variable(num_vars, boolean=True)
out_sub = cp.Variable(num_vars, bounds=[0, 1])

# 100 * (in_enable-in_sub - (out_enable - out_sub))
constraints = [
    (in_enable-in_sub) >= 0,
    (out_enable-out_sub) >= 0,
    100 * (in_enable-in_sub - (out_enable - out_sub)) == -1
    
]


objective = cp.sum(cp.abs(in_enable)) + cp.sum(cp.abs(out_enable))

p = cp.Problem(cp.Minimize(objective), constraints)

p.solve()
for v in [in_enable, in_sub, out_enable, out_sub]:
    print(v.value)

[0.]
[0.]
[1.]
[0.99]


In [14]:
import cvxpy as cp
from src.graph._lpProject import LpProject
from src.graph._lpScaledMatrix import LpScaledMatrix

def solveMinFreeVariables(project: LpProject):
    
    # additional_inputs = list(project.inputs - project.explicit_inputs)
    
    
    def var_dict_for(names, **kwargs):
        # var = cp.Variable(len(names), **kwargs) if len(names) > 0 else []
        var = cp.Variable(len(names), **kwargs) 
        # Fun fact, vars is not iterable but you can get items from it by index
        return var, dict(zip(names, [var[i] for i in range(len(names))]))
    
    recipe_vars, recipe_vars_dict = var_dict_for(project.recipe_names, name="recipes", nonneg=True)
    # explicit_input_amounts, explicit_input_amounts_dict = var_dict_for(project.explicit_inputs, name="expl_in", nonneg=True)
    in_switch, in_switch_dict = var_dict_for(project.variables, name="in_switch", boolean=True)
    in_sub, in_sub_dict = var_dict_for(project.variables, name="in_sub", bounds=[0, 1])
    out_switch, out_switch_dict = var_dict_for(project.variables, name="out_switch", boolean=True)
    out_sub, out_sub_dict = var_dict_for(project.variables, name="out_sub", bounds=[0, 1])

    # Target is the output of running one of each recipe (i.e. sum of each recipe vector)
    fake_target = np.sum(project.ing_matrix, axis=1)
    scaled_matrix = LpScaledMatrix(project.ing_matrix, fake_target)
    priority_ratio = scaled_matrix.max_min_ratio**4
    
    constraints = []
    
    # for item, recipe_coeffs in zip(project.variables, scaled_matrix.matrix):
        
        
    #     problem_eq = cp.sum(cp.multiply(recipe_vars, recipe_coeffs)) + input_term == 0
    #     constraints.append(problem_eq)
    
    # Item quantities must add up (for each item, item io * amount of recipe + input/output term == 0)
    constraints.append(
        cp.matmul(scaled_matrix.scaled_matrix, recipe_vars)
        + priority_ratio * (in_switch - in_sub) 
        - priority_ratio * (out_switch - out_sub) == 0)
    constraints.append((in_switch-in_sub) >= 0)
    constraints.append((out_switch-out_sub) >= 0)
    for explicit_input in project.explicit_inputs:
        constraints.append(in_switch_dict[explicit_input] == 1)
    
    # Every recipe must be used (here represented as >= 1)
    constraints.append(recipe_vars >= 1)
    
    
    objective = (
        -1 * (cp.sum(in_sub) + cp.sum(out_sub))
        + priority_ratio * (cp.sum(in_switch) + cp.sum(out_switch))
    )
    
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()
    print("Problem Status:", problem.status, "in", problem.compilation_time, "seconds")    
    
    # for v in [in_switch, in_sub, out_switch, out_sub]:
    #     print(v.value)
    print("\nINPUTS")
    for ing, sub, name in zip(np.array(in_switch.value), np.array(in_sub.value), project.variables):
        if ing==1: print(name, ing-sub)
    print("\nOUTPUTS")
    for out, sub, name in zip(np.array(out_switch.value), np.array(out_sub.value), project.variables):
        if out==1: print(name, out-sub)
    
    
    

# project = LpProject.fromConfig("devtest/baux_to_titanium.yaml")
# project = LpProject.fromConfig("devtest/bauxite.yaml")
# project = LpProject.fromConfig("223_monazite_line.yaml")
project = LpProject.fromConfig("devtest/palladium_loop.yaml")

problem_cvxpy = solveMinFreeVariables(project)